<a href="https://colab.research.google.com/github/meirarc/Market_Price/blob/main/Market_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
from __future__ import print_function
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json


In [94]:
enable_print = False
filename = "/content/drive/MyDrive/Colab/Market_Price.csv"

In [91]:
def print(*args, **kwargs):
  if enable_print:
    return __builtins__.print(*args, **kwargs)

In [85]:
def compare_price(current, new):
  print("(compare_price): current: ",current, " is blank: ", current == "")
  print("(compare_price): new: ", new, " is blank: ", new == "")
  print("(compare_price:) ",current, new)

  current = float(str(current).replace(",",".")) if current != "" and current is not None else 0 
  new = float(str(new).replace(",",".")) if new != "" and new is not None else 0

  status = ""
  status = "Same Price" if new == current else status  
  status = "More Expensive" if new > current else status   
  status = "On Sale" if new < current else status
  return status

In [43]:
def marche(item):
    webpage = requests.get(item["URL"])
    soup = BeautifulSoup(webpage.content, "html.parser")
    info = soup.find(attrs={"class":"name-and-info"}).p.get_text()
    
    product_name = soup.find(attrs={"class":"product-name"}).get_text()
    product_price = float(info.split("R$ ",1)[1].split("/",1)[0].replace(",","."))
    product_unit = info.split("R$ ",1)[1].split("/",1)[1]

    if product_unit.strip() == "kg":
      size = "1"
      price_kg = product_price
    else:
      size = product_name.split(" ")[-1]
      if size[-1] == "g":
        size = float(size.replace("g", ""))/1000
        price_kg = round(product_price/size, 2)
        product_unit = "g"
      else:
        if product_unit.strip() == "un":
          size = "3"
          price_kg = product_price * 3
        else:
          size = ""
          price_kg = ""

    status = compare_price(item["Product Price per Kg"], price_kg)
    
    item["Product Name"] = product_name if product_name is not None else ""
    item["Product Price"] = "{:.2f}".format(product_price) if product_price is not None else ""
    item["Product Unit"] = product_unit if product_unit is not None else ""
    item["Product Size"] = "{:.3f}".format(float(size)) if size is not None else ""
    item["Product Price per Kg"] = "{:.2f}".format(price_kg) if price_kg is not None else ""
    item["Status"] = status if status is not None else ""

    return item

In [45]:
def oba(item):

  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = ""
  product_price = ""
  product_unit = ""
  size = ""
  price_kg = ""

  product_name = soup.find(attrs={"class":re.compile("ProductDetailsStyles")})
  product_name = product_name if product_name is not None else ""
  
  product_price = soup.find(attrs={"class":re.compile("ProductDetailsStyles")})
  product_price = product_price if product_price is not None else ""
  
  product_unit = ""

  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""
  
  return item

In [46]:
def hakuo(item):
  
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = ""
  product_price = ""
  product_unit = ""
  size = ""
  price_kg = ""

  product_name = item["URL"].split("/")[-1].replace("-", " ")
  product_name = product_name if product_name is not None else ""
  
  product_price = soup.find(attrs={"class":re.compile("area-preco-detalhe-produto")})
  print(product_price)
  product_price = product_price if product_price is not None else ""
  
  product_unit = ""

  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""
  
  return item

In [95]:
def sams(item):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = soup.find(attrs={"class":re.compile("productBrand")})
  product_name = product_name.get_text().strip() if product_name is not None else "Error Product Name"
  print("(sams) product_name: ",product_name)

  product_price  = soup.find(attrs={"class":re.compile("productShowCasePricePdp")})
  product_price_kg  = soup.find(attrs={"class":re.compile("productShowCasePriceKgPdp")})
  
  price_kg = 0
  price = 0
  unit = ""
  
  if product_price_kg is not None:
    clean_price_kg = product_price_kg.get_text().strip()
    if clean_price_kg != "":
      price_kg = clean_price_kg.split(" ")[-1].replace(u'\xa0', ' ').split("$")[-1].replace(",",".").strip()
      unit = clean_price_kg.split(" ")[0]
  print("(sams) price_kg: ",price_kg)
  print("(sams) unit: ",unit)

  if product_price is not None:
    clean_price = product_price.get_text().strip()
    if clean_price != "":
      price = clean_price.split("$")[-1].replace(",",".").strip()
  print("(sams) price: ",price)

  product_price = price_kg if price_kg != 0 else price
  product_price = product_price if product_price is not None else ""
  print("(sams) product_price: ",product_price)

  product_unit = unit if unit != "" else ""
  product_unit = product_unit if product_unit is not None else ""
  print("(sams) product_unit: ", product_unit)

    
  if product_unit.strip() == "Preço/Kg":
    size = "1"
    price_kg = product_price
    product_unit = "kg"
  else:
    unit = product_name.strip().split(" ")[-1]
    product_unit = unit
    
    if product_unit == "Unidade":
      size = "3"
      price_kg = "{:.2f}".format(float(product_price) * 3) if product_price != "" else ""
      product_unit = "un" 
    else:
      if product_unit[-2:].lower() == "kg": 
        size = product_unit[:-2].replace(",", ".")
        price_kg = round(float(product_price) / float(size), 2)
        product_unit = "kg"
      else:
        print(product_unit)
        if product_unit.strip()[-1].lower() == "g": 
          size = float(product_unit[:-1])/1000
          price_kg = float(product_price) / size if product_price != "" else ""
          product_unit = "g"
        else:
          if product_name.strip() == "Batata Pacote":
              size = "2"
              price_kg = float(product_price) / 2
              product_unit = "kg"
          else:
            if product_name.strip() == "Agrião Verdureira":
              size = "3"
              price_kg = float(product_price) * 2
              product_unit = "un"
            else:
              size = ""
              price_kg = ""
              product_unit = ""

  print("(sams) current, item[Product Price per Kg]: ", item["Product Price per Kg"] == "")
  print("(sams) new, price_kg: ", price_kg)
  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""

  return item

In [68]:
def paodeacucar(item):

  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = soup.find(attrs={"class":"textstyles__TextComponent-w4b5ef-0 iISkjF"}).get_text().strip()
  product_unit = ""
  size = product_name.split(" ")[-1]

  product_price = soup.find(attrs={"class":re.compile("CurrentPrice")})
  product_price = float(product_price.p.get_text().split("R$ ",1)[1].replace(",",".")) if product_price is not None else ""


  if size[-2].lower() == "k":
    size = float(size.lower().replace("kg", "").replace(",","."))
    price_kg = round(product_price/size, 2) if product_price != "" else ""
    product_unit = "kg"
  else: 
    if size[-1] == "g":
      size = float(size.replace("g", ""))
      price_kg = round(product_price/size*1000, 2) if product_price != "" else ""
      product_unit = "g"
    else:
      size = "3"
      product_unit = "un"
      price_kg = product_price * 3

  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""

  return item

In [ ]:
df = pd.read_csv(filename, sep=";", encoding = "utf-8")
df["Product Price per Kg"] = df["Product Price per Kg"].str.replace(",",".")
df["Product Price"] = df["Product Price"].str.replace(",",".")
df["Product"] = df["Product"].str.replace(u'\xa0', u' ')
df = df.fillna("")
df

In [49]:
string = df.to_json(orient = 'records')
data = json.loads(string)

In [ ]:
for item in data:
  if item["URL"] is not None and item["URL"] != "":
    marche(item) if item["Market"] == "St. Marche" else None
    sams(item) if item["Market"] == "Sams Club" else None
    paodeacucar(item) if item["Market"] == "Pao de Acucar" else None
    #oba(item) if item["Market"] == "Oba" else None
    #hakuo(item) if item["Market"] == "Hakuo" else None

In [ ]:
data_df = pd.DataFrame(data)
data_df

In [ ]:
df.update(data_df)
df

In [73]:
df.to_csv(filename, index=False, sep=";", encoding = "utf-8")

In [ ]:
df.sort_values(by=["Market"]).query("Status == 'On Sale'")